In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [ ]:
# in this notebook, i will try to establish understanding of data, and implement logistic regression of data.


In [4]:
# some necessary libraries and packages
import sys
import matplotlib
import scipy as sp
import IPython 
from IPython import display
import sklearn

import random
import time



In [7]:
# load data modelling libraries

# modeling algorithm
from sklearn import linear_model

# model helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

# visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

#config the visualisers
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

In [20]:
data_raw = pd.read_csv('/kaggle/input/titanic/train.csv')
data_val = pd.read_csv('/kaggle/input/titanic/test.csv')

data1 = data_raw.copy(deep = True) # any cange in one, will not affect other. 

data_cleaner = [data1, data_val] # both train and test combined in 

print(data_raw.info())
data_raw.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
154,155,0,3,"Olsen, Mr. Ole Martin",male,NaN,0,0,Fa 265302,7.3125,NaN,S
244,245,0,3,"Attalah, Mr. Sleiman",male,30.0,0,0,2694,7.2250,NaN,C
299,300,1,1,"Baxter, Mrs. James (Helene DeLaudeniere Chaput)",female,50.0,0,1,PC 17558,247.5208,B58 B60,C
870,871,0,3,"Balkic, Mr. Cerin",male,26.0,0,0,349248,7.8958,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
37,38,0,3,"Cann, Mr. Ernest Charles",male,21.0,0,0,A./5. 2152,8.0500,NaN,S
482,483,0,3,"Rouse, Mr. Richard Henry",male,50.0,0,0,A/5 3594,8.0500,NaN,S
232,233,0,2,"Sjostedt, Mr. Ernst Adolf",male,59.0,0,0,237442,13.5000,NaN,S
205,206,0,3,"Strom, Miss. Telma Matilda",female,2.0,0,1,347054,10.4625,G6,S
129,130,0,3,"Ekstrom, Mr. Johan",male,45.0,0,0,347061,6.9750,NaN,S


In [17]:
#checking null elements in each feature
print('train columns with null values : \n', data1.isnull().sum())
print("_"*10)

print('test/val columns with null values : \n', data_val.isnull().sum())
print("_"*10)

data_raw.describe(include ='all')

train columns with null values : 
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
__________
test/val columns with null values : 
 PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64
__________


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Givard, Mr. Hans Kristensen",male,NaN,NaN,NaN,CA. 2343,NaN,C23 C25 C27,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [21]:
# impute the following columns
# Age with median 
# Embarked with Mode
# Fare with Median
for dataset in data_cleaner:
    dataset['Age'].fillna(dataset['Age'].median(), inplace= True)
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace= True)
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace= True)
    
drop_columns = ['PassengerId','Cabin','Ticket']
data1.drop(drop_columns, axis=1, inplace =True)

print(data1.isnull().sum())
print('_'*10)
print(data_val.isnull().sum())

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
__________
PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64


In [23]:
# craete new features, train and test/val data set
# family size -> no of members in family
# alone -> if was alone or not
# title -> includes Mr, Mrs, master .....
for dataset in data_cleaner :
    dataset['FamilySize'] = dataset['SibSp']+dataset['Parch'] + 1
    
    # init alone 
    dataset['IsAlone'] = 1
    dataset['IsAlone'].loc[dataset['FamilySize']>1] = 0 # ip values in  IsAlone
    
    # for title, str.split is used. 
    # Cumings, Mrs. John Bradley (Florence Briggs Thayer) ,is the entry
    # first split around ", "  and then select the second part
    # Mrs. John Bradley (Florence Briggs Thayer)
    # now split around "." and take the first part --> Mrs
    dataset['Title'] = dataset['Name'].str.split(", ",expand=True)[1].str.split(".",expand=True)[0]
    
    #craeting 4 farebins, ie. fare is now divided in 4 parts, qcut is used here,
    #because, it depends on freq, hence tries to have almost same no of data
    # in each bin. used in cases like below, where fares must be rather fixed already in a few slots. 
    dataset['FareBin'] = pd.qcut(dataset['Fare'], 4)
    
    #craeting bins for age . cut is used here and is generally used for Age. 
    # it creates bins without considering frequency of data, hence might have different 
    # no of data in different bins
    dataset['AgeBin'] = pd.cut(dataset['Age'].astype(int), 5)

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [25]:
data1['Title'].value_counts()
# as we can see here, to many titles are present, and may not prove to be helpful
# hence we will convert them to Misc (ie. miscillanious)

Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Mlle              2
Major             2
Col               2
Ms                1
the Countess      1
Jonkheer          1
Capt              1
Don               1
Lady              1
Mme               1
Sir               1
Name: Title, dtype: int64

In [29]:
stat_min =10
title_names = (data1['Title'].value_counts()<stat_min)
#print(title_names)
#print('_'*10)

data1['Title'] = data1['Title'].apply(lambda x: 'Misc'
                                     if title_names.loc[x]==True else x)
print(data1['Title'].value_counts())
print('_'*10)

Mr        517
Miss      182
Mrs       125
Master     40
Misc       27
Name: Title, dtype: int64
__________


In [30]:
# imputed datas !!!!
data1.info()
data_val.info()
data1.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Survived    891 non-null    int64   
 1   Pclass      891 non-null    int64   
 2   Name        891 non-null    object  
 3   Sex         891 non-null    object  
 4   Age         891 non-null    float64 
 5   SibSp       891 non-null    int64   
 6   Parch       891 non-null    int64   
 7   Fare        891 non-null    float64 
 8   Embarked    891 non-null    object  
 9   FamilySize  891 non-null    int64   
 10  IsAlone     891 non-null    int64   
 11  Title       891 non-null    object  
 12  FareBin     891 non-null    category
 13  AgeBin      891 non-null    category
dtypes: category(2), float64(2), int64(6), object(4)
memory usage: 85.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype   

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,Title,FareBin,AgeBin
49,0,3,"Arnold-Franchi, Mrs. Josef (Josefine Franchi)",female,18.0,1,0,17.8000,S,2,0,Mrs,"(14.454, 31.0]","(16.0, 32.0]"
5,0,3,"Moran, Mr. James",male,28.0,0,0,8.4583,Q,1,1,Mr,"(7.91, 14.454]","(16.0, 32.0]"
851,0,3,"Svensson, Mr. Johan",male,74.0,0,0,7.7750,S,1,1,Mr,"(-0.001, 7.91]","(64.0, 80.0]"
522,0,3,"Lahoud, Mr. Sarkis",male,28.0,0,0,7.2250,C,1,1,Mr,"(-0.001, 7.91]","(16.0, 32.0]"
72,0,2,"Hood, Mr. Ambrose Jr",male,21.0,0,0,73.5000,S,1,1,Mr,"(31.0, 512.329]","(16.0, 32.0]"
239,0,2,"Hunt, Mr. George Henry",male,33.0,0,0,12.2750,S,1,1,Mr,"(7.91, 14.454]","(32.0, 48.0]"
643,1,3,"Foo, Mr. Choong",male,28.0,0,0,56.4958,S,1,1,Mr,"(31.0, 512.329]","(16.0, 32.0]"
212,0,3,"Perkin, Mr. John Henry",male,22.0,0,0,7.2500,S,1,1,Mr,"(-0.001, 7.91]","(16.0, 32.0]"
651,1,2,"Doling, Miss. Elsie",female,18.0,0,1,23.0000,S,2,0,Miss,"(14.454, 31.0]","(16.0, 32.0]"
759,1,1,"Rothes, the Countess. of (Lucy Noel Martha Dye...",female,33.0,0,0,86.5000,S,1,1,Misc,"(31.0, 512.329]","(32.0, 48.0]"


In [ ]:
# now we will work on  --  converting formats !
# ie. convert catagorical data into mathematical data